In [5]:
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
import datetime as dt

import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_column', None)
tqdm.pandas()

# Motivation

In [6]:
df1 = pd.concat([pd.read_csv(f'MSAP_part{i}.csv', index_col=0) for i in range(1, 4)])

In [7]:
df1.isnull().sum(axis=0)

ID_Operation_History           0
DATE_REPORT                    0
DATE_REPORT_SRC                0
DATE_MONTH                     0
EMPLOYEE_FIO              962347
EMPLOYEE_ID              3008687
EMPLOYEE_POSITION        3009508
EBIRTH                   3027042
RBEGDA                   3015450
RENDDA                   3015450
TB_ID                          0
urf_code_uni                2728
SOURCE_SYSTEM             357210
OPERATION                  21881
QUANTITY                     172
CHANNEL_GR               3174465
IS_FRAUD                       0
DESCRIPTION               450765
insert_date               450765
IN_MOTIV                       0
PRODUCT_NAME               21881
vl_up                    5172088
DATE_UPDATE                    0
PAYMENT_STATUS           8551378
epk_id                         0
untb                       47429
fullname                   47429
birth                      47429
Табельный_клиента       13219207
Отпуск                  12365398
dtype: int

In [8]:
df1.dtypes

ID_Operation_History      int64
DATE_REPORT              object
DATE_REPORT_SRC          object
DATE_MONTH               object
EMPLOYEE_FIO             object
EMPLOYEE_ID             float64
EMPLOYEE_POSITION        object
EBIRTH                   object
RBEGDA                   object
RENDDA                   object
TB_ID                     int64
urf_code_uni             object
SOURCE_SYSTEM            object
OPERATION                object
QUANTITY                 object
CHANNEL_GR               object
IS_FRAUD                  int64
DESCRIPTION              object
insert_date              object
IN_MOTIV                  int64
PRODUCT_NAME             object
vl_up                    object
DATE_UPDATE              object
PAYMENT_STATUS           object
epk_id                    int64
untb                    float64
fullname                 object
birth                    object
Табельный_клиента       float64
Отпуск                   object
dtype: object

### Убираем нулевые значения и переводим в нужный формат

In [10]:
df1.loc[(df1['RENDDA'] > '2023-01-01'), 'RENDDA'] = '2262-1-1'

In [11]:
df1['EBIRTH'] = df1['EBIRTH'].fillna('1700-1-1')
df1['RBEGDA'] = df1['RBEGDA'].fillna('1700-1-1')
df1['RENDDA'] = df1['RENDDA'].fillna('2262-1-1')
df1['insert_date'] = df1['insert_date'].fillna('1700-1-1')
df1['DATE_UPDATE'] = df1['DATE_UPDATE'].fillna('1700-1-1')


In [12]:
df1 = df1.fillna({
    'EMPLOYEE_FIO':'-',
    'EMPLOYEE_POSITION': '-',
    'urf_code_uni': '-',
    'SOURCE_SYSTEM': '-',
    'OPERATION': '-',
    'QUANTITY': 1,
    'CHANNEL_GR': '-',
    'DESCRIPTION': '-',
    'PRODUCT_NAME': '-',
    'vl_up': 0,
    'fullname': '~'
})

In [13]:
df1['DATE_REPORT'] = pd.to_datetime(df1['DATE_REPORT']).dt.date
df1['EBIRTH'] = pd.to_datetime(df1['EBIRTH']).dt.date
df1['RBEGDA'] = pd.to_datetime(df1['RBEGDA']).dt.date
df1['RENDDA'] = pd.to_datetime(df1['RENDDA']).dt.date
df1['insert_date'] = pd.to_datetime(df1['insert_date']).dt.date
df1['DATE_UPDATE'] = pd.to_datetime(df1['DATE_UPDATE']).dt.date
df1['birth'] = pd.to_datetime(df1['birth']).dt.date
df1['DATE_MONTH'] = pd.to_datetime(df1['DATE_MONTH'])
#df1 = df1.astype({'untb':'Int64', 'EMPLOYEE_ID': 'Int64'})

### Смотрим некоторые категориальные признаки и их значения

In [12]:
df1['EMPLOYEE_POSITION'].value_counts()

СТАРШИЙ КЛИЕНТСКИЙ МЕНЕДЖЕР                 6734647
КЛИЕНТСКИЙ МЕНЕДЖЕР                         1020731
СТАРШИЙ МЕНЕДЖЕР ПО ОБСЛУЖИВАНИЮ             819626
РУКОВОДИТЕЛЬ ОФИСА                           813090
ЗАМЕСТИТЕЛЬ РУКОВОДИТЕЛЯ ОФИСА               283180
КОНСУЛЬТАНТ                                  175356
КЛИЕНТСКИЙ МЕНЕДЖЕР ПРЕМЬЕР                  116166
МЕНЕДЖЕР ПО СЕРВИСУ                           72622
ФИНАНСОВЫЙ МЕНЕДЖЕР                           60537
СТ.МЕНЕДЖЕР ПО ИПОТЕЧНОМУ КРЕДИТОВАНИЮ        39027
ВЕДУЩИЙ МЕНЕДЖЕР ПО ОБСЛУЖИВАНИЮ              33678
МЕНЕДЖЕР ПО ИПОТЕЧНОМУ КРЕДИТОВАНИЮ           33375
КОНСУЛЬТАНТ ПО РАБОТЕ С ПФР                   33294
СПЕЦИАЛИСТ                                    27207
СТАРШИЙ СПЕЦИАЛИСТ                            17194
СТАРШИЙ МЕНЕДЖЕР ПО СЕРВИСУ                   17117
МЕНЕДЖЕР ПО СДЕЛКАМ С НЕДВИЖИМОСТЬЮ           14696
СТ. МЕНЕДЖЕР ПО СДЕЛКАМ С НЕДВИЖИМОСТЬЮ       12922
ОПЕРАЦИОННЫЙ МЕНЕДЖЕР                         12239
МЕНЕДЖЕР ПО 

In [26]:
df1['SOURCE_SYSTEM'].unique()

array(['АС МБК', 'АС ЕРИБ', 'АС Транзакт', 'АС ФС', 'АС СПООБК', 'АС ЦОД',
       nan, 'Билинг Сбербанк-Телекома', 'АС ЕФС', 'АС БС', 'АС ЕКП',
       'АС ЕПС', 'САМОКАТ', 'СБЕРБОКС', 'ОККО', 'OSAGO', 'Сбермаркет',
       'Реестры', 'ЦБДБО', 'CRM-Корпоративный', 'АС ЕКС+АС SM',
       'Страховки ДЗО', 'СберУслуги', 'ДомКлик', 'BUSINESS_CARD',
       'CRM_RKO_CHANGE'], dtype=object)

In [27]:
df1['OPERATION'].unique()

array(['Оформление подключения к услуге ',
       'Оформление: Выписка по счету ДК/вклада/сбер счета/КК, Реквизиты, Выписка для госслужащих, Справка о доступном остатке, об уплаченных % ',
       'Платеж по карте/счету (через СБОЛ) ',
       'Прием документов на получение кредита ',
       'Оформление заявления на выдачу банковских карт ',
       'Оформление заявления на выдачу дебетовой банковской карты МИР Золотая зарплатная сотруднику бюджетного учреждения ',
       'Выдача банковской карты ', 'Обновление данных клиентов ',
       'Платеж по карте/счету (через FSB) ',
       'Оформление заявления на выдачу личной дебетовой карты Сберкарта через СБОЛ ',
       'Оформление ежемесячной подписки "СберПрайм +" ',
       'Открытие вклада / ОМС ',
       'Оформление и оплата годовой подписки "Сберпрайм" ',
       'Оформление вклада On-line ',
       'Оформление заявления на перевод заработной платы на счет, открытый в Сбербанке ',
       'Реактивация Мобильного приложения СберБанк Онлайн +

In [24]:
df1['CHANNEL_GR'].unique()

array(['ВСП', nan, 'ПРЕМЬЕР', 'ПРЯМЫЕ ПРОДАЖИ', 'ПАРТНЁРЫ',
       'ПРЯМЫЕ ПРОДАЖИ(ЦОПП)', 'ВИП', 'ЭКВАЙРИНГ'], dtype=object)

In [36]:
df1['PRODUCT_NAME'].value_counts()

Потребительские кредиты (кроме кредита под залог недвижимости)                                                                                3141868
Банковские карты                                                                                                                               976918
Мобильный банк                                                                                                                                 866144
Кредитные карты (стандартные)                                                                                                                  833873
Дебетовые карты без овердрафта (зарплатные)                                                                                                    769727
Дебетовая карта Сберкарта                                                                                                                      712669
Платеж по карте/счету                                                                               

### Добавляем некоторые сводные столбцы по количеству операций за период:

In [13]:
df1 = df1.merge(df1[['DATE_REPORT','EMPLOYEE_ID', 'ID_Operation_History']].groupby(['EMPLOYEE_ID','DATE_REPORT'], as_index=False).\
    agg({'ID_Operation_History':'count'}).rename(columns={'ID_Operation_History':'emp_oper_per_day'}), on=['DATE_REPORT','EMPLOYEE_ID'], how='left')
df1 = df1.merge(df1[['DATE_REPORT','untb', 'ID_Operation_History']].groupby(['untb','DATE_REPORT'], as_index=False).\
    agg({'ID_Operation_History':'count'}).rename(columns={'ID_Operation_History':'client_product_per_day'}), on=['DATE_REPORT','untb'], how='left')

In [14]:
time_series = df1[['DATE_REPORT', 'ID_Operation_History', 'EMPLOYEE_ID', 'untb']]
time_series['DATE_REPORT'] = pd.to_datetime(df1['DATE_REPORT'])
#time_series = time_series.set_index('DATE_REPORT')
time_series['month'] = time_series['DATE_REPORT'].dt.month
time_series['year'] = time_series['DATE_REPORT'].dt.year
df1['month'] = df1['DATE_MONTH'].dt.month
df1['year'] = df1['DATE_MONTH'].dt.year

In [11]:
df1 = df1.merge(time_series.groupby(['EMPLOYEE_ID', 'year', 'month'], as_index=False)[['ID_Operation_History']].count().\
          rename(columns={'ID_Operation_History':'emp_oper_per_month'}), on=['EMPLOYEE_ID', 'year', 'month'], how='left')

In [15]:
time_series.head()

,DATE_REPORT,ID_Operation_History,EMPLOYEE_ID,untb,month,year
0,2021-08-09,25664113,961677.0,26822468.0,8,2021
1,2021-08-18,25664134,1800908.0,21381134.0,8,2021
2,2021-08-13,25664157,1116607.0,20020401.0,8,2021
3,2021-08-17,25664169,1889942.0,11772029.0,8,2021
4,2021-08-23,25664176,389534.0,2986391.0,8,2021


### Создаём dataset для обучения и добавляем в него признаки

In [16]:
df = pd.DataFrame() #dataset

In [18]:
df['ID_Operation_History'] = df1['ID_Operation_History']
df['date_report'] = pd.to_datetime(df1['DATE_REPORT']).dt.day
df['emploee_position'] = df1['EMPLOYEE_POSITION']
df['emp_age'] = (df1['DATE_REPORT'] - df1['EBIRTH']) / dt.timedelta(days=365)
df['report-start_job'] = (df1['DATE_REPORT'] - df1['RBEGDA']).dt.days
df['finish_job-report'] = (df1['RENDDA'] - df1['DATE_REPORT']).dt.days
df['SOURCE_SYSTEM'] = df1['SOURCE_SYSTEM']
df['OPERATION'] = df1['OPERATION']
df['CHANNEL_GR'] = df1['CHANNEL_GR']
df['urf_code_uni'] = df1['urf_code_uni']
df['QUANTITY'] = df1['QUANTITY']
df['DESCRIPTION'] = df1['DESCRIPTION']
df['IN_MOTIV'] = df1['IN_MOTIV']
df['PRODUCT_NAME'] = df1['PRODUCT_NAME']
df['vl_up'] = df1['vl_up']
df['insert-report'] = (df1['insert_date'] - df1['DATE_REPORT']).dt.days
df['update-report'] = (df1['DATE_UPDATE'] - df1['DATE_REPORT']).dt.days
df['client_age'] = (df1['DATE_REPORT'] - df1['birth']) / dt.timedelta(days=365)
df['client_is_emploee'] = np.where(df1['Табельный_клиента'].isnull(), 0, 1)
df['emploee_in_vacation'] = np.where(df1['Отпуск'].isnull(), 0, 1)
df['emp_oper_per_day'] = df1['emp_oper_per_day']
df['emp_oper_per_month'] = df1['emp_oper_per_month']
df['day_of_week'] = pd.to_datetime(df1['DATE_REPORT']).dt.weekday_name
df['client_product_per_day'] = df1['client_product_per_day']
df['target'] = df1['IS_FRAUD']

In [20]:
df['target'] = df1['IS_FRAUD']

In [25]:
#df.to_csv('dataset_1.csv')

In [ ]:
df = pd.read_csv('dataset_1.csv')

Не добавленный признак с первого раза, нужно переписать

In [ ]:
time_series['client_product_at_moment'] = time_series[['DATE_REPORT', 'untb']].progress_apply(lambda x: time_series[(time_series['DATE_REPORT'] < x[0])
                                                                          &(time_series['untb'] == x[1])]['ID_Operation_History'].count(), axis=1)

  0%|          | 0/13397810 [00:00<?, ?it/s]

In [ ]:
#df['client_product_at_moment']= time_series['client_product_at_moment']

### Количество уже имеющихся продуктов у человека на момент продажи нового продукта за 181 день:

In [138]:
clprod = pd.read_csv('client_product_since20210101.csv', sep=';')
clprod['DATE_REPORT'] = pd.to_datetime(clprod['DATE_REPORT'])

In [139]:
clprod = clprod.groupby(['DATE_REPORT','untb'], as_index=False).agg({'ID_Operation_History':'count'})

In [140]:
day_oper = time_series.groupby(['DATE_REPORT','untb'], as_index=False).agg({'ID_Operation_History':'count'})

In [141]:
day_oper['DATE_REPORT'][0] - dt.datetime(2021, 1, 1, 0, 0,0)

Timedelta('181 days 00:00:00')

In [142]:
clprod.dtypes

DATE_REPORT             datetime64[ns]
untb                             int64
ID_Operation_History             int64
dtype: object

In [143]:
dates = day_oper['DATE_REPORT'].drop_duplicates()

In [144]:
day_oper = pd.concat([clprod, day_oper])

In [145]:
day_oper

,DATE_REPORT,untb,ID_Operation_History
0,2021-01-01,173475.0,1
1,2021-01-01,174525.0,1
2,2021-01-01,175934.0,1
3,2021-01-01,177232.0,1
4,2021-01-01,177648.0,1
...,...,...,...
8050384,2021-12-31,47341427.0,4
8050385,2021-12-31,47341437.0,6
8050386,2021-12-31,47341445.0,6
8050387,2021-12-31,47346703.0,2


In [146]:
dates.iloc[0] - dt.timedelta(181)

Timestamp('2021-01-01 00:00:00')

In [147]:
df1['DATE_REPORT'] = pd.to_datetime(df1['DATE_REPORT'])
#client_product = df1[['ID_Operation_History', 'DATE_REPORT', 'untb']].merge(oper_data, on=['DATE_REPORT', 'untb'], how='inner')

In [148]:
oper_data = pd.DataFrame()
oper_per_data = pd.DataFrame()
for date in tqdm(dates):
    oper_per_data = day_oper[(day_oper['DATE_REPORT'] < date)&(day_oper['DATE_REPORT'] >= date - dt.timedelta(181))].\
                           groupby('untb', as_index=False).agg({'ID_Operation_History':'sum'})
    oper_per_data['DATE_REPORT'] = date
    oper_per_data = df1[['ID_Operation_History','untb']][df1['DATE_REPORT'] == date].merge(oper_per_data, on='untb', how='inner')
    oper_data = pd.concat([oper_data, oper_per_data])

  0%|          | 0/184 [00:00<?, ?it/s]

In [149]:
oper_data

,ID_Operation_History_x,untb,ID_Operation_History_y,DATE_REPORT
0,19812303,46400017.0,5,2021-07-01
1,20942271,46400017.0,5,2021-07-01
2,19823247,46400017.0,5,2021-07-01
3,19867023,46400017.0,5,2021-07-01
4,19871127,46400017.0,5,2021-07-01
...,...,...,...,...
16222,52691206,9813184.0,2,2021-12-31
16223,52826735,33881746.0,6,2021-12-31
16224,52925755,30270978.0,2,2021-12-31
16225,52931048,32924220.0,3,2021-12-31


In [150]:
oper_data = oper_data.rename(columns={'ID_Operation_History_y': 'client_product_at_moment', 'ID_Operation_History_x': 'ID_Operation_History'})

In [151]:
oper_data.to_csv('product_at_moment.csv')

Старый способ (ещё более долгий):

In [13]:
time_series[(time_series['DATE_REPORT'] < pd.datetime(2021, 8, 13))&(time_series['untb'] == 20020401.0)]['ID_Operation_History'].count()

2

# Emploee info & client info

In [19]:
emp = pd.read_csv('Emp_info.csv', sep=';')

In [20]:
client = pd.read_csv('client_info.csv', sep=';')

#### Emploee

In [21]:
emp = emp[['EMPLOYEE_FIO', 'EBIRTH', 'emploee_ActAddress', 'emploee_BirthPlace',
       'emploee_DateUnseeing', 'emploee_DeathDay', 'emploee_FLAG_AGENT',
       'emploee_FLAG_EXIST_AGENT', 'emploee_Literate',
       'emploee_PENSIONER_DATE', 'emploee_Resident', 'emploee_Sex',
       'emploee_Unseeing']].drop_duplicates()

In [22]:
emp['emploee_DeathDay'] = emp['emploee_DeathDay'].fillna('1800-01-01')
emp['emploee_PENSIONER_DATE'] = emp['emploee_PENSIONER_DATE'].fillna('2262-01-01')
emp['emploee_Sex'] = emp['emploee_Sex'].fillna(0)
emp['EBIRTH'] = emp['EBIRTH'].fillna('1700-1-1')
emp['EMPLOYEE_FIO'] = emp['EMPLOYEE_FIO'].fillna('-')
emp['emploee_FLAG_AGENT'] = emp['emploee_FLAG_AGENT'].fillna(0)
emp['emploee_FLAG_EXIST_AGENT'] = emp['emploee_FLAG_EXIST_AGENT'].fillna(0)

In [23]:
emp['EBIRTH'] = pd.to_datetime(emp['EBIRTH']).dt.date
emp['emploee_DeathDay'] = pd.to_datetime(emp['emploee_DeathDay']).dt.date
emp['emploee_PENSIONER_DATE'] = pd.to_datetime(emp['emploee_PENSIONER_DATE']).dt.date

In [24]:
df1 = df1.merge(emp[['EMPLOYEE_FIO', 'EBIRTH', 'emploee_DeathDay', 'emploee_FLAG_AGENT',
       'emploee_FLAG_EXIST_AGENT', 'emploee_PENSIONER_DATE', 'emploee_Resident', 'emploee_Sex']], on=['EMPLOYEE_FIO', 'EBIRTH'], how='left')

In [38]:
df = df.merge(df1[['ID_Operation_History', 'emploee_DeathDay', 'emploee_FLAG_AGENT',
       'emploee_FLAG_EXIST_AGENT', 'emploee_PENSIONER_DATE', 'emploee_Resident', 'emploee_Sex']], on='ID_Operation_History', how='left')

In [25]:
emp.head()

,EMPLOYEE_FIO,EBIRTH,emploee_ActAddress,emploee_BirthPlace,emploee_DateUnseeing,emploee_DeathDay,emploee_FLAG_AGENT,emploee_FLAG_EXIST_AGENT,emploee_Literate,emploee_PENSIONER_DATE,emploee_Resident,emploee_Sex,emploee_Unseeing
0,Абазян Яна Анатольевна,1990-03-15,692342!ПРИМОРСКИЙ!!!7!АРСЕНЬЕВ!44!ОКТЯБРЬСКАЯ!...,ГОР. АРСЕНЬЕВ ПРИМОРСКИЙ КРАЙ,1800-01-01,1800-01-01,0.0,0.0,1.0,2262-01-01,1,2.0,2.0
1,Абаимова Мария Васильевна,1990-07-11,"443000 Г. САМАРА, УЛ. СОЛНЕЧНАЯ Д. 15 КВ. 123",ГОР.КУЙБЫШЕВ,1800-01-01,1800-01-01,0.0,0.0,NaN,2262-01-01,1,2.0,NaN
2,Абакумов Александр Сергеевич,1985-02-03,629307!ЯМАЛО-НЕНЕЦКИЙ!!!7!НОВЫЙ УРЕНГОЙ!48!ГУБ...,Г ОМСК,1800-01-01,1800-01-01,0.0,0.0,1.0,2262-01-01,1,1.0,2.0
11,Абакумова Анаита Сергеевна,1988-11-10,628602!АО ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ...,С. СТРОКИНО КОЛОСОВСКИЙ РАЙОН ОМСКАЯ ОБЛАСТЬ,1800-01-01,1800-01-01,0.0,0.0,1.0,2262-01-01,1,2.0,2.0
15,Абакумова Анаита Сергеевна,1988-11-10,644080!ОБЛ ОМСКАЯ!05!СЕДЕЛЬНИКОВСКИЙ!10!СЕДЕЛЬ...,С. СТРОКИНО КОЛОСОВСКИЙ РАЙОН ОМСКАЯ ОБЛАСТЬ,1800-01-01,1800-01-01,0.0,0.0,1.0,2262-01-01,1,2.0,2.0


In [26]:
emp[['EMPLOYEE_FIO', 'EBIRTH']].drop_duplicates()

,EMPLOYEE_FIO,EBIRTH
0,Абазян Яна Анатольевна,1990-03-15
1,Абаимова Мария Васильевна,1990-07-11
2,Абакумов Александр Сергеевич,1985-02-03
11,Абакумова Анаита Сергеевна,1988-11-10
17,Абакумова Елена Викторовна,1989-10-12
18,Абакумова Елена Владимировна,1971-10-21
21,Абакумова Наталья Сергеевна,1990-02-27
25,Абалова Татьяна Алексеевна,1996-01-07
29,Абалтусова Вера Вениаминовна,1969-02-18
32,Абанина Любовь Васильевна,1961-07-23


In [27]:
client = client.drop(['DEPOSIT_Balance', 'DEPOSIT_BranchNo', 'DEPOSIT_Office', 'DEPOSIT_OpCash', 'DEPOSIT_OpenDay', 'DEPOSIT_OpNo'], axis=1).drop_duplicates()

In [28]:
client['PERSON_DeathDay'] = client['PERSON_DeathDay'].fillna('1800-01-01')
client['PERSON_PENSIONER_DATE'] = client['PERSON_PENSIONER_DATE'].fillna('2262-01-01')
client['PERSON_Sex'] = client['PERSON_Sex'].fillna(0)
client['PERSON_FLAG_AGENT'] = client['PERSON_FLAG_AGENT'].fillna(0)
client['PERSON_FLAG_EXIST_AGENT'] = client['PERSON_FLAG_EXIST_AGENT'].fillna(0)

In [29]:
client['PERSON_Resident'].unique()

array([ 1.0000e+00,  2.0000e+00,  0.0000e+00,         nan, -2.9152e+04])

In [30]:
key = pd.read_csv('Key_for_client.csv', sep=';')

In [31]:
client = key.merge(client[['Номер_счёта_клиента','PERSON_DeathDay', 'PERSON_FLAG_AGENT', 'PERSON_FLAG_EXIST_AGENT',
                  'PERSON_PENSIONER_DATE', 'PERSON_Resident', 'PERSON_Sex']], on='Номер_счёта_клиента', how='left')

In [32]:
client = client.drop('Номер_счёта_клиента', axis=1)

In [33]:
df = df.merge(client, on='ID_Operation_History', how='left')

In [34]:
client.head()

,ID_Operation_History,PERSON_DeathDay,PERSON_FLAG_AGENT,PERSON_FLAG_EXIST_AGENT,PERSON_PENSIONER_DATE,PERSON_Resident,PERSON_Sex
0,22824803,1800-01-01,0.0,0.0,2262-01-01,1.0,2.0
1,22836703,1800-01-01,0.0,0.0,2262-01-01,1.0,2.0
2,21824382,1800-01-01,0.0,0.0,2262-01-01,1.0,2.0
3,21824382,1800-01-01,0.0,0.0,2262-01-01,1.0,2.0
4,22836793,1800-01-01,0.0,0.0,2262-01-01,1.0,1.0


# FIN 

In [108]:
clemp = pd.read_csv('Cl_to_emp.csv', sep=';',thousands=',')

In [109]:
clemp

,ID_Operation_History,Номер_счёта_клиента,emploee_PRINTABLENO,OPER_DATE,OPER_AMT
0,20704900,40817810032003530693,40817810532008969989,2020-01-01,1000
1,20938828,40817810032003530693,40817810532008969989,2020-01-01,1000
2,21377956,40817810032003530693,40817810532008969989,2020-01-01,1000
3,42800836,40817810032003530693,40817810532008969989,2020-01-01,1000
4,42819988,40817810032003530693,40817810532008969989,2020-01-01,1000
...,...,...,...,...,...
397447,43538223,40817810432003536077,40817810432001278463,2021-12-31,1000
397448,44914299,40817810372003263603,40817810472004097804,2021-12-31,4000
397449,45692479,40817810532001233162,40817810032001233251,2021-12-31,154
397450,46664965,40817810916547152578,40817810516541606789,2021-12-31,100


In [110]:
clemp = clemp.merge(df1[['ID_Operation_History', 'DATE_REPORT']], on='ID_Operation_History', how='left')

In [111]:
clemp['DATE_REPORT'] = pd.to_datetime(clemp['DATE_REPORT'])
clemp['OPER_DATE'] = pd.to_datetime(clemp['OPER_DATE'])
clemp['OPER_AMT'] = clemp['OPER_AMT'].astype('float')

In [113]:
clemp['clemp_diff_dates_before'] = clemp['DATE_REPORT'] - clemp['OPER_DATE']
clemp['clemp_diff_dates_after'] = clemp['OPER_DATE'] - clemp['DATE_REPORT']

In [114]:
clemp.loc[clemp['clemp_diff_dates_before'] < dt.timedelta(0), ['clemp_diff_dates_before']] = dt.timedelta(100_000)
clemp.loc[clemp['clemp_diff_dates_after'] < dt.timedelta(0), ['clemp_diff_dates_after']] = dt.timedelta(100_000)

In [115]:
clemp.dtypes

ID_Operation_History                 int64
Номер_счёта_клиента                 object
emploee_PRINTABLENO                 object
OPER_DATE                   datetime64[ns]
OPER_AMT                           float64
DATE_REPORT                 datetime64[ns]
diff_dates_before          timedelta64[ns]
diff_dates_after           timedelta64[ns]
clemp_diff_dates_before    timedelta64[ns]
clemp_diff_dates_after     timedelta64[ns]
dtype: object

In [116]:
clemp = clemp[['ID_Operation_History', 'OPER_AMT', 'clemp_diff_dates_before', 'clemp_diff_dates_after']].\
        groupby('ID_Operation_History', as_index=False).agg({'OPER_AMT':'sum', 'clemp_diff_dates_before':'min', 'clemp_diff_dates_after': 'min'})

In [119]:
clemp.rename(columns={'OPER_AMT':'clemp_amt'}).to_csv('clemp_data.csv')

Аналогично с переводами сотрудник - клиенту

In [120]:
empcl = pd.read_csv('emp_to_cl.csv', sep=';',thousands=',')

In [121]:
empcl

,ID_Operation_History,Номер_счёта_клиента,emploee_PRINTABLENO,OPER_DATE,OPER_AMT
0,19785287,40817810872002705840,40817810772003681322,2020-06-01,10000
1,19793495,40817810872002705840,40817810772003681322,2020-06-01,10000
2,19851677,40817810316545275280,40817810616545275032,2020-06-01,5000
3,19882978,40817810572333900339,40817810772008497799,2020-06-01,1000
4,19918108,40817810806003188176,40817810606000446817,2020-06-01,4300
...,...,...,...,...,...
448723,20244529,40817810306003391311,40817810306006094770,2021-09-14,300
448724,24029897,40817810967450152068,40817810467450297397,2021-09-14,500
448725,42021721,40817810306003391311,40817810306006094770,2021-09-14,300
448726,42174950,40817810406002877820,40817810506008903216,2021-09-14,202


In [122]:
empcl = empcl.merge(df1[['ID_Operation_History', 'DATE_REPORT']], on='ID_Operation_History', how='left')

In [123]:
empcl['DATE_REPORT'] = pd.to_datetime(empcl['DATE_REPORT'])
empcl['OPER_DATE'] = pd.to_datetime(empcl['OPER_DATE'])
empcl['OPER_AMT'] = empcl['OPER_AMT'].astype('float')

In [124]:
empcl['empcl_diff_dates_before'] = empcl['DATE_REPORT'] - empcl['OPER_DATE']
empcl['empcl_diff_dates_after'] = empcl['OPER_DATE'] - empcl['DATE_REPORT']

In [125]:
empcl.loc[empcl['empcl_diff_dates_before'] < dt.timedelta(0), ['empcl_diff_dates_before']] = dt.timedelta(100_000)
empcl.loc[empcl['empcl_diff_dates_after'] < dt.timedelta(0), ['empcl_diff_dates_after']] = dt.timedelta(100_000)

In [130]:
empcl.dtypes

ID_Operation_History                 int64
Номер_счёта_клиента                 object
emploee_PRINTABLENO                 object
OPER_DATE                   datetime64[ns]
OPER_AMT                           float64
DATE_REPORT                 datetime64[ns]
empcl_diff_dates_before    timedelta64[ns]
empcl_diff_dates_after     timedelta64[ns]
dtype: object

In [131]:
empcl = empcl[['ID_Operation_History', 'OPER_AMT', 'empcl_diff_dates_before', 'empcl_diff_dates_after']].\
        groupby('ID_Operation_History', as_index=False).agg({'OPER_AMT':'sum', 'empcl_diff_dates_before':'min', 'empcl_diff_dates_after': 'min'})

In [132]:
empcl

,ID_Operation_History,OPER_AMT,empcl_diff_dates_before,empcl_diff_dates_after
0,19246082,10.0,287 days,100000 days
1,19251554,10.0,291 days,100000 days
2,19256560,6100.0,213 days,100000 days
3,19292271,10.0,100000 days,75 days
4,19293073,1400.0,184 days,100000 days
...,...,...,...,...
53356,53225688,63887.0,109 days,100000 days
53357,53225834,1000.0,377 days,100000 days
53358,53225912,63887.0,109 days,100000 days
53359,53226640,174.0,478 days,100000 days


In [133]:
empcl.rename(columns={'OPER_AMT':'empcl_amt'}).to_csv('empcl_data.csv')